In [141]:
import numpy as np
import os
import cv2

def get_sift(path):

    sift = cv2.xfeatures2d.SIFT_create()

    desc ={}
    images=[]
    c=[]
    keys = []
    alldes = np.empty([1,128])
    for objdir in os.listdir(path):
        obj = path + objdir
        des=[]
        for img in os.listdir(obj):
            img1 = obj + '/' + img
            images.append(img1)
            img1 = cv2.imread(img1)
            img1= img1[:300,250:400,:]
            kp1,des1 = sift.detectAndCompute(img1,None)
            keys.append(kp1)
            des.append(des1)
            c.append(des1.shape[0])
            alldes = np.concatenate([alldes,des1])
            print('{}'.format(img),end="\r")
        #keys[objdir] = kp
        desc[objdir] = des
        print('{} done'.format(objdir))
    alldesc_real = alldes[1:,:]
    return keys,c,images,desc,alldesc_real



In [911]:
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
import os
import math
import pickle

In [231]:
len(alldesc)

338819

In [27]:
path = os.getcwd()
path = path[:path.rfind('/')] + '/train/'

In [142]:
kp,count,images,des, alldesc = get_sift(path)

3m_high_tack_spray_adhesive done
aunt_jemima_original_syrup done
campbells_chicken_noodle_soup done
cheez_it_white_cheddar done
cholula_chipotle_hot_sauce done
clif_crunch_chocolate_chip done
coca_cola_glass_bottle done
detergent done
expo_marker_red done
listerine_green done
nice_honey_roasted_almonds done
nutrigrain_apple_cinnamon done
palmolive_green done
pringles_bbq done
vo5_extra_body_volumizing_shampoo done
vo5_split_ends_anti_breakage_shampoo done


In [146]:
np.savez('desc.npz',des)
np.savez('count.npz',count)
np.savez('sift_features.npz',alldesc)
np.savez('images.npz',images)
#dump(kp,'kp.joblib')

In [6]:
images = np.load('images.npz')['arr_0']
count = np.load('count.npz')['arr_0']

In [917]:
kpfile = open('keypoints.p','wb')
pickle.dump(clust,kpfile)

In [920]:
f = open('keypoints.p','r')
c2 = pickle.load(f)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte

In [16]:
from joblib import dump,load
import numpy as np
clust = load('clusters_12000.joblib')

In [10]:
x
clust.cluster_centers_[clust.predict(np.array([x]))]
euclidean_distances(clust.cluster_centers_[clust.predict(np.array([x]))],np.array([x]))[0][0]

NameError: name 'x' is not defined

In [445]:
max_dist=-float('inf')

sift = cv2.xfeatures2d.SIFT_create()

for img in images:
    img1 = cv2.imread(img)
    img1= img1[:300,250:400,:]
    kp1,des1 = sift.detectAndCompute(img1,None)
    for x in des1:
        #cl = clust.predict(np.array([x]))
        dist = euclidean_distances(clust.cluster_centers_[clust.predict(np.array([x]))],np.array([x]))[0][0]
        if dist > max_dist:
            max_dist = dist
print(max_dist)
        

388.6596797495266


In [584]:
words = np.shape(clust.cluster_centers_)[0]
N = 3456
w = np.zeros(words)
idf = np.zeros([N,words])

In [909]:
N

3456

In [961]:
indoc = np.zeros(words)
nd = np.zeros(N)
ni = np.zeros(words)
nid = np.zeros([N,words])
i=0
d=0
sift = cv2.xfeatures2d.SIFT_create()
for k in range(len(images)):
    for j in range(count[d]):
        word = clust.labels_[i]
        if kp[d][j].pt[0] > 50 and kp[d][j].pt[0] < 280:
            if kp[d][j].pt[1] > 10 and kp[d][j].pt[1] < 120:
                nid[d][word] += 100
            else:
                nid[d][word] += 1
        else:
            nid[d][word] += 1
        if indoc[word]==0:
            indoc[word]=1
            ni[word]+=1
            #nd[d]+=1
        #nid[d][word]+=1
        i+=1
    d+=1
    indoc = np.zeros(words)

In [ ]:
for obj in os.listdir(path):
    for img in des[obj]:
        for disc in range(np.shape(img)[0]):
            word = clust.labels_[i]
            if indoc[word]==0:
                indoc[word]=1
                ni[word]+=1
                nd[d]+=1
            nid[d][word]+=1
            #print(nid[d][word])
            i+=1
        d+=1
        indoc = np.zeros(words)

In [962]:
for d in range(N):
    nd[d]= sum(nid[d])
for d in range(N):
    nid[d] = nid[d]/nd[d]

In [989]:
weights = np.zeros([N,words])

for d in range(N):
    for i in range(words):
        weights[d][i] = (nid[d][i]) * np.log(N/ni[i])
        #print(weights[d][i])

In [904]:
np.savez('tfidf.npz',weights)

In [905]:
we2 = np.load('tfidf.npz')['arr_0']

In [25]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from scipy.spatial.distance import cosine

In [522]:
test_path = path[:path.rfind('/')]
test_path = test_path[:test_path.rfind('/')] + '/sample_test/'
test_path

'/mnt/Stuff/Acads/CS783/A1/sample_test/'

In [142]:
sift = cv2.xfeatures2d.SIFT_create()
for img in os.listdir(test_path):
    if img == 'instances.txt':
        continue
    imr = test_path + img
    test = cv2.imread(imr)
    kpt, dest = sift.detectAndCompute(test,None)
    query=np.zeros(words)
    j = clust.predict(dest)
    for x in j:
        query[x]+=1
    break
query

array([ 0.,  3., 20.,  1.,  0.,  0.,  0.,  2.,  2.,  0.,  2.,  0.,  0.,
        0.,  2.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  2.,  0.,  1.,  0.,
        0.,  1.,  3.,  1.,  4.,  8.,  0.,  1.,  0.,  1.,  0.,  0.,  5.,
        0.,  0.,  0.,  2.,  1.,  7.,  2.,  2.,  0.,  0.,  1.,  0.,  0.,
        1.,  0.,  0.,  0.,  1.,  0.,  0.,  3.,  2.,  0.,  0., 12.,  0.,
        0.,  1.,  0.,  5.,  0.,  1.,  1.,  0.,  0.,  1.,  0.,  1.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
        2.,  1.,  2.,  0.,  1.,  0.,  1.,  2.])

In [1006]:
max_dist=20000

In [1033]:
sift = cv2.xfeatures2d.SIFT_create(sigma=1.4)
#img = 'campbells_chicken_noodle_soup/N2_69.jpg'
img = 'test_1.jpg'
imr = test_path + img
test = cv2.imread(imr)
#test = test[:300,250:400,:]
kpt, dest = sift.detectAndCompute(test,None)
query=np.zeros(words)
print(dest.shape)
real=0
for x in dest:
    dist = euclidean_distances(clust.cluster_centers_[clust.predict(np.array([x]))],np.array([x]))[0][0]
    if dist < max_dist:
        real+=1
        query[clust.predict(np.array([x]))] +=1
print(real)       
#for x in j:
#    query[x]+=1

(434, 128)
434


In [1034]:
sim=[]
for i in range(N):
    l = cosine(np.array([query]),weights[i])
    sim.append(l)

In [1035]:
y = np.argsort(sim)
#for i in range(N):
 #   print(y[i])

In [1036]:
ans=0
for i in range(100):
    out = images[y[i]][32:images[y[i]].rfind('/')] + '_' + images[y[i]][images[y[i]].rfind('/')+1:]
    print(out)
    if images[y[i]][32:images[y[i]].rfind('/')] == 'listerine_green':
        ans+=1
print(ans)

vo5_extra_body_volumizing_shampoo_N1_333.jpg
expo_marker_red_N1_303.jpg
campbells_chicken_noodle_soup_N1_312.jpg
nutrigrain_apple_cinnamon_N1_291.jpg
cheez_it_white_cheddar_N1_276.jpg
vo5_extra_body_volumizing_shampoo_N1_330.jpg
3m_high_tack_spray_adhesive_N1_0.jpg
vo5_split_ends_anti_breakage_shampoo_N1_333.jpg
vo5_split_ends_anti_breakage_shampoo_N1_339.jpg
pringles_bbq_N1_333.jpg
pringles_bbq_N1_324.jpg
campbells_chicken_noodle_soup_N1_321.jpg
3m_high_tack_spray_adhesive_N1_357.jpg
vo5_split_ends_anti_breakage_shampoo_N1_327.jpg
expo_marker_red_N1_309.jpg
cholula_chipotle_hot_sauce_N1_327.jpg
3m_high_tack_spray_adhesive_N1_6.jpg
nice_honey_roasted_almonds_N1_342.jpg
palmolive_green_N1_339.jpg
vo5_extra_body_volumizing_shampoo_N1_336.jpg
expo_marker_red_N1_312.jpg
cholula_chipotle_hot_sauce_N1_321.jpg
3m_high_tack_spray_adhesive_N1_3.jpg
vo5_split_ends_anti_breakage_shampoo_N1_336.jpg
clif_crunch_chocolate_chip_N1_300.jpg
expo_marker_red_N1_300.jpg
palmolive_green_N1_333.jpg
cholula_

In [37]:
cat instances.txt
















hard_multi_3.jpg campbells_chicken_noodle_soup,pringles_bbq

In [209]:
for objdir in os.listdir(path):
    print(objdir)

3m_high_tack_spray_adhesive
aunt_jemima_original_syrup
campbells_chicken_noodle_soup
cheez_it_white_cheddar
cholula_chipotle_hot_sauce
clif_crunch_chocolate_chip
coca_cola_glass_bottle
detergent
expo_marker_red
listerine_green
nice_honey_roasted_almonds
nutrigrain_apple_cinnamon
palmolive_green
pringles_bbq
vo5_extra_body_volumizing_shampoo
vo5_split_ends_anti_breakage_shampoo
